# Table of Contents<a id="top"></a>
- [Importing Libraries](#import)
- [Importing Scraped Data](#data)
- [Count Vectorizer Models - No Stemming/Lemmetization, Stop Words Removed](#cvec1)
- [TFIDF Vectorizer Models - No Stemming/Lemmetization, Stop Words Removed](#tvec1)
- [Function to Tokenize, Lemmatize and Stem Posts](#func)
- [Count Vectorizer Models - Lemmetized, Stop Words Removed](#cvec2)
- [TFIDF Vectorizer Models - Lemmetized, Stop Words Removed](#tvec2)
- [Count Vectorizer Models - Stemmed, Stop Words Removed](#cvec3) <-- Best Performing Vectorized Model
- [TFIDF Vectorizer Models - Stemmed, Stop Words Removed](#tvec3)
- [Naive Bayes Models](#nb)
- [KNN, Random Forrest, SVM](#other)

### Importing Libraries <a id="import"></a>

In [361]:
# Import Libraries

import pandas as pd
import numpy as np

# Library tools to turn text in to interpretable DataFrames
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer 
# Object that uses count vectorizer and Logistic Regrssion as one
from sklearn.pipeline import Pipeline
# Split data to check train model, Input parameters to create best model
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction import stop_words
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

# Importing lemmatizer.
import nltk
from nltk.stem import WordNetLemmatizer

# Importing stemmer.
from nltk.stem.porter import PorterStemmer

# Import Tokenizer
from nltk.tokenize import RegexpTokenizer

# Naive Bayes Models 
from sklearn.naive_bayes import MultinomialNB, GaussianNB

### Importing Scraped Data <a id="data"></a>

In [133]:
text_df = pd.read_csv('./text_df.csv')
text_df.drop('Unnamed: 0', axis=1, inplace=True)

In [202]:
text_df['text'][10]

'The Unofficial Rewatch Thread - S3E02 "Run" **From TV Guide:** Rebecca defends ACN again as another lawsuit looms; Neal could be in trouble after a dangerous leak; Charlie and Leona confront a hostile takeover attempt by Reese\'s half-siblings; Sloan worries that Don has crossed an ethical line; Hallie regrets a late-night tweet; Maggie weighs the pros and cons of eavesdropping.\n\n**From IMDb:** While Rebecca must once again defend ACN during a possible lawsuit, Will tries to protect Neal from the aftermath of the DOD leak; Charlie and Leona deal with a hostile takeover; Sloan worries about Don\'s involvement with insider information.\n\n**Original Discussion Thread [HERE](https://www.reddit.com/r/Thenewsroom/comments/2mips3/episode_discussion_s03e02_run/)**'

### Count Vectorizer Models - No Stemming/Lemmetization, Stop Words Removed <a id="cvec1"></a>

In [129]:
# Setting parameters
X = text_df['text']
y = text_df['class']

In [130]:
# Train test split data

# Because the sample size is large and I noticed characters were being missed as indicators if they didn't make it 
# in to the training data, set test_size = 0.1
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=30)

In [131]:
# Instantiate Pipeline for Count Vectorizer
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [132]:
# Run Grid Search to create optimum  model
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8338907469342252


{'cvec__max_df': 0.9,
 'cvec__max_features': 2500,
 'cvec__min_df': 3,
 'cvec__ngram_range': (1, 1)}

In [52]:
# ReRun Grid Search based on previous results
pipe_params = {
    'cvec__max_features': [2900, 3000, 3100],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.87, .9, .93],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.846711259754738


{'cvec__max_df': 0.87,
 'cvec__max_features': 2900,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [53]:
# ReRun Grid Search based on previous results
pipe_params = {
    'cvec__max_features': [2700, 2800, 2900],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.87],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=5)

gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8450390189520625


{'cvec__max_df': 0.87,
 'cvec__max_features': 2900,
 'cvec__min_df': 3,
 'cvec__ngram_range': (1, 2)}

In [54]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Model is overfit. How to fix? 
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.979933110367893.
Test score: 0.865.


In [55]:
# Creating data frames to better understand the false predictions

train_df = pd.DataFrame()
train_df['text'] = X_train
train_df['actual_y'] = y_train
train_df['pred_y'] = y_pred_train

test_df = pd.DataFrame()
test_df['text'] = X_test
test_df['actual_y'] = y_test
test_df['pred_y'] = y_pred_test

In [201]:
# Out of curiosity - Reading the incorrect predictions in the overfit train data. 
train_df[train_df.pred_y != train_df.actual_y]
test_df[test_df.pred_y != test_df.actual_y]


KeyError: 10

In [57]:
# Checking to see what words are showing up. 

cvec_all_text = CountVectorizer(ngram_range=(1,2), max_features=3700, min_df=2, max_df=0.87, stop_words='english')

cvec_all_text.fit(X_train)

X_train_cv = cvec_all_text.transform(X_train)

X_train_cv = pd.DataFrame(X_train_cv.toarray(),
                                 columns = cvec_all_text.get_feature_names())

X_train_cv.sum().sort_values(ascending=False).head(15)

just         588
season       543
episode      487
like         416
don          326
think        326
newsroom     299
know         298
sorkin       283
president    278
charlie      274
time         268
west         256
wing         252
west wing    251
dtype: int64

### TFIDF Vectorizer Models - No Stemming/Lemmetization, Stop Words Removed <a id="tvec1"></a>

[return to top](#top)

In [58]:
# Instantiate Pipeline for TFID Vectorizer 
pipe = Pipeline([('tvec', TfidfVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [61]:
pipe_params = {
    'tvec__max_features': [2500, 3000, 3500],
    'tvec__min_df': [2, 3],
    'tvec__max_df': [.7, .8, .9],
    'tvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8651059085841695


{'tvec__max_df': 0.7,
 'tvec__max_features': 3000,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [63]:
pipe_params = {
    'tvec__max_features': [2900, 3000, 3100],
    'tvec__min_df': [2, 3],
    'tvec__max_df': [.1, .3, .7],
    'tvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8651059085841695


{'tvec__max_df': 0.3,
 'tvec__max_features': 3000,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [64]:
pipe_params = {
    'tvec__max_features': [3000],
    'tvec__min_df': [2],
    'tvec__max_df': [.2, .3, .4],
    'tvec__ngram_range': [(1,2), (1,3)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8651059085841695


{'tvec__max_df': 0.3,
 'tvec__max_features': 3000,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [65]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Model is overfit. How to fix? 
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.9537346711259754.
Test score: 0.875.


### Function to Tokenize, Lemmatize and Stem Posts<a id="func"></a>

[return to top](#top)

In [213]:
# Tokenize, lemmatizing, and stemming function 
import time

# Instantiating
tokenizer = RegexpTokenizer(r'\w+')
lemmatizer = WordNetLemmatizer()
p_stemmer = PorterStemmer()

def lem_stem(df):
    start = time.time()
    print(start)
    # Adding columns to DataFrame
    df['lem_text'] = None
    df['stem_text'] = None
    
    # Looping through each post
    for i in range(df.shape[0]):
        
        post = df['text'][i]
        tokens = tokenizer.tokenize(post.lower()) 
        
        new_post_lem = ''
        new_post_stem = ''
        
        for token in tokens:
            lem = lemmatizer.lemmatize(token)
            stem = p_stemmer.stem(token)
            new_post_lem += ' ' + lem
            new_post_stem += ' ' + stem
            
        df['lem_text'][i] = new_post_lem
        df['stem_text'][i] = new_post_stem
        
        if i%100 == 0:
            print(f'{i} posts complete.')
    end = time.time()
    print(end - start)
    return df


1562809726.7048068


In [214]:
# Commenting out calling this code because it was extremely time intensive and I do not want to run it again 
# by mistake. If I could spend more time on this project, I would like to find a way to do this that is less
# computationally intensive.

# lem_stem(text_df)
# text_df.to_csv('./lem_stem.csv')

# Saving csv and loading below

1562809763.522989
0 posts complete.


/Users/mags/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/mags/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100 posts complete.
200 posts complete.
300 posts complete.
400 posts complete.
500 posts complete.
600 posts complete.
700 posts complete.
800 posts complete.
900 posts complete.
1000 posts complete.
1100 posts complete.
1200 posts complete.
1300 posts complete.
1400 posts complete.
1500 posts complete.
1600 posts complete.
1700 posts complete.
1800 posts complete.
1900 posts complete.
1893.0269360542297


,text,class,lem_text,stem_text
0,The Unofficial Rewatch Thread - S3E06: “What K...,0,the unofficial rewatch thread s3e06 what kind...,the unoffici rewatch thread s3e06 what kind o...
1,Just started watching it and have a few questi...,0,just started watching it and have a few quest...,just start watch it and have a few question i...
2,The Unofficial Rewatch Thread - S3E05: “Oh She...,0,the unofficial rewatch thread s3e05 oh shenan...,the unoffici rewatch thread s3e05 oh shenando...
3,"On the bus, early season 2 Added the spoiler t...",0,on the bus early season 2 added the spoiler t...,on the bu earli season 2 ad the spoiler tag j...
4,The Unofficial Rewatch Thread: S3E04 - “Contem...,0,the unofficial rewatch thread s3e04 contempt ...,the unoffici rewatch thread s3e04 contempt fr...
5,What to watch after my 4th rewatch? It is one ...,0,what to watch after my 4th rewatch it is one ...,what to watch after my 4th rewatch it is one ...
6,Don Keefer's Greatest Hits,0,don keefer s greatest hit,don keefer s greatest hit
7,"The Unofficial Rewatch Thread - S3E03 ""Main Ju...",0,the unofficial rewatch thread s3e03 main just...,the unoffici rewatch thread s3e03 main justic...
8,What welfare programs don’t work? Not sure on ...,0,what welfare program don t work not sure on t...,what welfar program don t work not sure on th...
9,What did Will mean when he said he believed in...,0,what did will mean when he said he believed i...,what did will mean when he said he believ in ...


In [211]:
start

NameError: name 'start' is not defined

In [146]:
lem_stem_df = pd.read_csv('./lem_stem.csv')
lem_stem_df.drop('Unnamed: 0', axis=1, inplace=True)

### Count Vectorizer Models - Lemmetized, Stop Words Removed <a id="cvec2"></a>

[return to top](#top)

In [149]:
X = lem_stem_df['lem_text']
y = lem_stem_df['class']

In [150]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=30)

In [151]:
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [152]:
pipe_params = {
    'cvec__max_features': [2500, 3000, 3500],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.9, .95],
    'cvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8311036789297659


{'cvec__max_df': 0.9,
 'cvec__max_features': 3000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [154]:
pipe_params = {
    'cvec__max_features': [2900, 3000, 3100],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.75, .85, .9],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8311036789297659


{'cvec__max_df': 0.75,
 'cvec__max_features': 3000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [159]:
pipe_params = {
    'cvec__max_features': [3000],
    'cvec__min_df': [2],
    'cvec__max_df': [.15, .35, .75],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8355629877369007


{'cvec__max_df': 0.15,
 'cvec__max_features': 3000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [161]:
pipe_params = {
    'cvec__max_features': [3000],
    'cvec__min_df': [2],
    'cvec__max_df': [.14, .15, .16],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8355629877369007


{'cvec__max_df': 0.15,
 'cvec__max_features': 3000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [162]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Model accuracy is almost identical to non lemmetized model.
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.9788182831661093.
Test score: 0.855.


### TFIDF Vectorizer Models - Lemmetized, Stop Words Removed <a id="tvec2"></a>

[return to top](#top)

In [163]:
pipe = Pipeline([('tvec', TfidfVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [167]:
pipe_params = {
    'tvec__max_features': [2000, 3000],
    'tvec__min_df': [2],
    'tvec__max_df': [.2, .3, .4],
    'tvec__ngram_range': [(1,2), (1,3)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8584169453734671


{'tvec__max_df': 0.3,
 'tvec__max_features': 3000,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [170]:
pipe_params = {
    'tvec__max_features': [2900, 3000, 3100],
    'tvec__min_df': [2, 3],
    'tvec__max_df': [.25, .3],
    'tvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8606465997770345


{'tvec__max_df': 0.25,
 'tvec__max_features': 3100,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [171]:
pipe_params = {
    'tvec__max_features': [3100, 3200, 3300],
    'tvec__min_df': [2],
    'tvec__max_df': [.1, .2, .25],
    'tvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8606465997770345


{'tvec__max_df': 0.25,
 'tvec__max_features': 3100,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [172]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Model accuracy is almost identical to non lemmetized model, just slightly better
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.9559643255295429.
Test score: 0.875.


### Count Vectorizer Models - Stemmed, Stop Words Removed <a id="cvec3"></a>

[return to top](#top)

In [419]:
X = lem_stem_df['stem_text']
y = lem_stem_df['class']

In [420]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=30)

In [421]:
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [177]:
pipe_params = {
    'cvec__max_features': [2000, 3000, 4000],
    'cvec__min_df': [2, 3],
    'cvec__max_df': [.05, .15, .3],
    'cvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8327759197324415


{'cvec__max_df': 0.3,
 'cvec__max_features': 4000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [179]:
pipe_params = {
    'cvec__max_features': [3500, 4000, 4500],
    'cvec__min_df': [2],
    'cvec__max_df': [.23, .3, .35],
    'cvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8333333333333334


{'cvec__max_df': 0.23,
 'cvec__max_features': 4000,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2)}

In [422]:
pipe_params = {
    'cvec__max_features': [4050],
    'cvec__min_df': [2],
    'cvec__max_df': [.22, .23, .24],
    'cvec__ngram_range': [(1,2)],
    'lr__C' : [.08],
    'lr__penalty': ['l2']
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8472686733556298


{'cvec__max_df': 0.23,
 'cvec__max_features': 4050,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2),
 'lr__C': 0.08,
 'lr__penalty': 'l2'}

In [395]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Best version of three very similar models
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.947603121516165.
Test score: 0.88.


In [423]:
preds = pd.DataFrame(y_test)

In [425]:
preds['lr'] = gs.predict(X_test)

### TFIDF Vectorizer Models - Stemmed, Stop Words Removed <a id="tvec3"></a>

[return to top](#top)

In [184]:
pipe = Pipeline([('tvec', TfidfVectorizer(stop_words='english')),
                 ('lr', LogisticRegression())])

In [185]:
pipe_params = {
    'tvec__max_features': [2600, 3100, 3600],
    'tvec__min_df': [2, 3],
    'tvec__max_df': [.2, .25, .3],
    'tvec__ngram_range': [(1,1), (1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8578595317725752


{'tvec__max_df': 0.3,
 'tvec__max_features': 3100,
 'tvec__min_df': 3,
 'tvec__ngram_range': (1, 2)}

In [188]:
pipe_params = {
    'tvec__max_features': [2700, 2800, 2950],
    'tvec__min_df': [3],
    'tvec__max_df': [.27, .28, .29],
    'tvec__ngram_range': [(1,2)]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

/Users/mags/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.8578595317725752


{'tvec__max_df': 0.29,
 'tvec__max_features': 2950,
 'tvec__min_df': 3,
 'tvec__ngram_range': (1, 2)}

In [189]:
print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

# Identical performance to lemmetized data
y_pred_test = gs.predict(X_test)
y_pred_train = gs.predict(X_train)

Train score: 0.955406911928651.
Test score: 0.875.


### Naive Bayes Model <a id="nb"></a>

[return to top](#top)

#### Models to run
I will be creating 6 total Naive Bayes models to compare based on my best found hyperparameters for each of the 6 types of Vectorized models that I ran. I will be running MultinomialNB on my count vectorizer models and GaussianNB on my TFIDF models.

- Count Vectorizer - No Stemming/Lemmetization, Stop Words Removed
    - {'cvec__max_df': 0.87, 'cvec__max_features': 2900, 'cvec__min_df': 3, 'cvec__ngram_range': (1, 2)}
    - Train accuracy score = 98.0%
    - Test accuracy score = 86.5%
- TFIDF Vectorizer - No Stemming/Lemmetization, Stop Words Removed
    - {'tvec__max_df': 0.3, 'tvec__max_features': 3000, 'tvec__min_df': 2, 'tvec__ngram_range': (1, 2)}
    - Train accuracy score = 95.3%
    - Test accuracy score = 87.5%
- Count Vectorizer - Lemmetized, Stop Words Removed
    - {'cvec__max_df': 0.15, 'cvec__max_features': 3000, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2)}
    - Train accuracy score = 97.9%
    - Test accuracy score = 85.5%
- TFIDF Vectorizer - Lemmetized, Stop Words Removed
    - {'tvec__max_df': 0.25, 'tvec__max_features': 3100, 'tvec__min_df': 2, 'tvec__ngram_range': (1, 2)}
    - Train accuracy score = 95.6%
    - Test accuracy score = 87.5%
- Count Vectorizer - Stemmed, Stop Words Removed <-- Best Performing Vectorized Model
    - {'cvec__max_df': 0.23, 'cvec__max_features': 4100, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 2)}
    - Train accuracy score = 98.5%
    - Test accuracy score = 87.5%
- TFIDF Vectorizer - Stemmed, Stop Words Removed
    - {'tvec__max_df': 0.29, 'tvec__max_features': 2950, 'tvec__min_df': 3, 'tvec__ngram_range': (1, 2)}
    - Train accuracy score = 95.5%
    - Test accuracy score = 87.5%

In [196]:
X = lem_stem_df['text']
y = lem_stem_df['class']

vect = CountVectorizer(max_df=0.87, max_features=2900, ngram_range=(1,2), stop_words='english')
vect.fit(X)

X_transform = vect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

nb = MultinomialNB()
nb.fit(X_train, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, nb.predict(X_train))}')
print(f'Test score: {metrics.accuracy_score(y_test, nb.predict(X_test))}')

Train score: 0.9241917502787068
Test score: 0.885


In [198]:
tvect = TfidfVectorizer(max_df=0.3, max_features=3000, min_df=2, ngram_range=(1,2), stop_words='english')
tvect.fit(X)

X_transform = tvect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

gb = GaussianNB()
gb.fit(X_train_dense, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, gb.predict(X_train_dense))}')
print(f'Test score: {metrics.accuracy_score(y_test, gb.predict(X_test_dense))}')

Train score: 0.9437012263099219
Test score: 0.775


In [199]:
X = lem_stem_df['lem_text']
y = lem_stem_df['class']

vect = CountVectorizer(max_df=0.15, max_features=3000, min_df=2, ngram_range=(1,2), stop_words='english')
vect.fit(X)

X_transform = vect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

nb = MultinomialNB()
nb.fit(X_train, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, nb.predict(X_train))}')
print(f'Test score: {metrics.accuracy_score(y_test, nb.predict(X_test))}')

Train score: 0.9319955406911928
Test score: 0.885


In [200]:
tvect = TfidfVectorizer(max_df=0.25, max_features=3100, min_df=2, ngram_range=(1,2), stop_words='english')
tvect.fit(X)

X_transform = tvect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

gb = GaussianNB()
gb.fit(X_train_dense, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, gb.predict(X_train_dense))}')
print(f'Test score: {metrics.accuracy_score(y_test, gb.predict(X_test_dense))}')

Train score: 0.9453734671125975
Test score: 0.755


In [426]:
X = lem_stem_df['stem_text']
y = lem_stem_df['class']

vect = CountVectorizer(max_df=0.23, max_features=4100, min_df=2, ngram_range=(1,2), stop_words='english')
vect.fit(X)

X_transform = vect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

nb = MultinomialNB()
nb.fit(X_train, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, nb.predict(X_train))}')
print(f'Test score: {metrics.accuracy_score(y_test, nb.predict(X_test))}')

Train score: 0.9375696767001115
Test score: 0.885


In [427]:
preds['nb'] = nb.predict(X_test)

In [204]:
tvect = TfidfVectorizer(max_df=0.29, max_features=2950, min_df=3, ngram_range=(1,2), stop_words='english')
tvect.fit(X)

X_transform = tvect.transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_transform, y, stratify=y, test_size=0.1, random_state=30)

X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

gb = GaussianNB()
gb.fit(X_train_dense, y_train)

print(f'Train score: {metrics.accuracy_score(y_train, gb.predict(X_train_dense))}')
print(f'Test score: {metrics.accuracy_score(y_test, gb.predict(X_test_dense))}')

Train score: 0.9509476031215162
Test score: 0.74


### KNN, Random Forrest, SVM <a id="other"></a>

[return to top](#top)

In [428]:
X = lem_stem_df['stem_text']
y = lem_stem_df['class']

In [429]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.1, random_state=30)

In [430]:
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('knn', KNeighborsClassifier())])

In [431]:
pipe_params = {
    'cvec__max_features': [50],
    'cvec__min_df': [2],
    'cvec__max_df': [.34],
    'cvec__ngram_range': [(1,2)],
    'knn__n_neighbors': [21],
    'knn__metric': ['euclidean']
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)
gs.best_params_

print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

0.7157190635451505
Train score: 0.7385730211817169.
Test score: 0.72.


In [432]:
preds['knn'] = gs.predict(X_test)

In [433]:
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('rf', RandomForestClassifier())])

In [434]:
pipe_params = {
    'cvec__max_features': [2000],
    'cvec__min_df': [3],
    'cvec__max_df': [.8],
    'cvec__ngram_range': [(1,2)],
    'rf__n_estimators': [700], 
    'rf__max_depth' : [40],
    'rf__min_samples_split': [200], 
    'rf__min_samples_leaf': [3]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)


print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')
gs.best_params_

0.8539576365663322
Train score: 0.9041248606465998.
Test score: 0.845.


{'cvec__max_df': 0.8,
 'cvec__max_features': 2000,
 'cvec__min_df': 3,
 'cvec__ngram_range': (1, 2),
 'rf__max_depth': 40,
 'rf__min_samples_leaf': 3,
 'rf__min_samples_split': 200,
 'rf__n_estimators': 700}

In [435]:
preds['rf'] = gs.predict(X_test)

In [436]:
pipe = Pipeline([('cvec', CountVectorizer(stop_words='english')),
                 ('svc', SVC())])

In [437]:
pipe_params = {
    'cvec__max_features': [300],
    'cvec__min_df': [2],
    'cvec__max_df': [.3],
    'cvec__ngram_range': [(1,2)],
    'svc__degree' : [0],
    'svc__C' : [1, 3 ],
    'svc__gamma' : [.001, .01],
    'svc__kernel' : ['rbf']
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)


print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

gs.best_params_

0.798216276477146
Train score: 0.8857302118171684.
Test score: 0.845.


{'cvec__max_df': 0.3,
 'cvec__max_features': 300,
 'cvec__min_df': 2,
 'cvec__ngram_range': (1, 2),
 'svc__C': 3,
 'svc__degree': 0,
 'svc__gamma': 0.01,
 'svc__kernel': 'rbf'}

In [438]:
preds['svc'] = gs.predict(X_test)

In [439]:
pipe = Pipeline([('tvec', TfidfVectorizer(stop_words='english')),
                 ('knn', KNeighborsClassifier())])

In [440]:
pipe_params = {
    'tvec__max_features': [41],
    'tvec__min_df': [2],
    'tvec__max_df': [.25],
    'tvec__ngram_range': [(1,2)],
    'knn__n_neighbors': [19],
    'knn__metric': ['euclidean'],
    'knn__leaf_size' : [10]
}
gs = GridSearchCV(pipe, param_grid=pipe_params, cv=3)
gs.fit(X_train, y_train)
print(gs.best_score_)

print(f'Train score: {gs.score(X_train, y_train)}.')
print(f'Test score: {gs.score(X_test, y_test)}.')

gs.best_params_

0.7363433667781494
Train score: 0.7536231884057971.
Test score: 0.745.


{'knn__leaf_size': 10,
 'knn__metric': 'euclidean',
 'knn__n_neighbors': 19,
 'tvec__max_df': 0.25,
 'tvec__max_features': 41,
 'tvec__min_df': 2,
 'tvec__ngram_range': (1, 2)}

In [441]:
preds['knn2'] = gs.predict(X_test)

In [442]:
probs = preds.drop('class', axis=1).apply(lambda x: np.mean(x), axis=1)

In [443]:
preds['avg'] = probs>0.5

In [444]:
preds['avg'] = preds['avg'].map({False:0, True:1})

In [445]:
metrics.accuracy_score(preds['class'], preds['avg'])

0.865